# From Wow-Effect to Production: Building Reliable LLM Systems

## Introduction

The most famous applications of LLMs are the ones that I like to call the "wow effect LLMs." There are plenty of viral LinkedIn posts about them, and they all sound like this:

"I built [x] that does [y] in [z] minutes using AI." 

Where: 

- **[x]** is usually something like a web app/platform
- **[y]** is a somewhat impressive main feature of [x]
- **[z]** is usually an integer number between 5 and 10
- **"AI"** is really, most of the time, a LLM wrapper (Cursor, Codex, or similar)

If you notice carefully, the focus of the sentence is not really the quality of the analysis but the amount of time you save. This is to say that, when dealing with a task, people are not excited about the LLM output quality in tackling the problem, but they are thrilled that the LLM is spitting out something quick that might sound like a solution to their problem.

This is why I refer to them as **wow-effect LLMs**. As impressive as they sound and look, these wow-effect LLMs display multiple issues that prevent them from being actually implemented in a production environment. Some of them:

- **The prompt is usually not optimized**: you don't have time to test all the different versions of the prompts, evaluate them, and provide examples in 5-10 minutes.

- **They are not meant to be sustainable**: in that short of time, you can develop a nice-looking plug-and-play wrapper. By default, you are throwing all the costs, latency, maintainability, and privacy considerations out of the window. 

- **They usually lack context**: LLMs are powerful when they are plugged into a big infrastructure, they have decisional power over the tools that they use, and they have contextual data to augment their answers. No chance of implementing that in 10 minutes. 

Now, don't get me wrong: LLMs are designed to be intuitive and easy to use. This means that evolving LLMs from the wow effect to production level is not rocket science. However, it requires a specific methodology that needs to be implemented. 

**The goal of this blog post is to provide this methodology.**

The points we will cover to move from wow-effect LLMs to production-level LLMs are the following:

1. **LLM System Requirements** - When this beast goes into production, we need to know how to maintain it. This is done in stage zero, through adequate system requirements analysis.  

2. **Prompt Engineering** - We are going to optimize the prompt structure and provide some best-practice prompt strategies.

3. **Force structure with schemas and structured output** - We are going to move from free text to structured objects, so the format of your response is fixed and reliable.

4. **Use tools so the LLM does not work in isolation** - We are going to let the model connect to data and call functions. This provides richer answers and reduces hallucinations.

5. **Add guardrails and validation around the model** - Check inputs and outputs, enforce business rules, and define what happens when the model fails or goes out of bounds.

6. **Combine everything into a simple, testable pipeline** - Orchestrate prompts, tools, structured outputs, and guardrails into a single flow that you can log, monitor, and improve over time.

We are going to use a very simple case: **we are going to make an LLM grade data science tests**. This is just a concrete case to avoid a totally abstract and confusing article. The procedure is general enough to be adapted to other LLM applications, typically with very minor adjustments.

Looks like we've got a lot of ground to cover. Let's get started!


## Tough Choices: Cost, Latency, Privacy

Before writing any code, there are a few important questions to ask:

**How complex is your task?**  
Do you really need the latest and most expensive model, or can you use a smaller one or an older family?

**How often do you run this, and at what latency?**  
Is this a web app that must respond on demand, or a batch job that runs once and stores results? Do users expect an immediate answer, or is "we'll email you later" acceptable?

**What is your budget?**  
You should have a rough idea of what is "ok to spend". Is it 1k, 10k, 100k? And compared to that, would it make sense to train and host your own model, or is that clearly overkill?

**What are your privacy constraints?**  
Is it ok to send this data through an external API? Is the LLM seeing sensitive data? Has this been approved by whoever owns legal and compliance?

For simple tasks, where you have a low budget and need low latency, the smaller models (for example the 4.x mini family or 5 nano) are usually your best bet. They are optimized for speed and price, and for many basic use cases like classification, tagging, light transformations, or simple assistants, you will barely notice the quality difference while paying a fraction of the cost.

For more complex tasks, such as complex code generation, long-context analysis, or high-stakes evaluations, it can be worth using a stronger model in the 5.x family, even at a higher per-token cost. In those cases, you are explicitly trading money and latency for better decision quality.

If you are running large offline workloads, for example re-scoring or re-evaluating thousands of items overnight, batch endpoints can significantly reduce costs compared to real-time calls. This often changes which model fits your budget, because you can afford a "bigger" model when latency is not a constraint.

From a privacy standpoint, it is also good practice to only send non-sensitive or "sensitive-cleared" data to your provider, meaning data that has been cleaned to remove anything confidential or personal. If you need even more control, you can consider running local LLMs.


## A LLM Teacher: The Grading Task

For this article, we're building an **automated grading system for Data Science exams**. Students take a test that requires them to analyze actual datasets and answer questions based on their findings. The LLM's job is to grade these submissions by:

1. Understanding what each question asks
2. Accessing the correct answers and grading criteria
3. Verifying student calculations against the actual data
4. Providing detailed feedback on what went wrong

This is a perfect example of why LLMs need tools and context. Without access to the datasets and grading rubrics, the LLM cannot grade accurately. It needs to retrieve the actual data to verify whether a student's answer is correct.

### The Test Structure

Our exam is stored in `test.json` and contains 10 questions across three sections. Students must analyze three different datasets: e-commerce sales, customer demographics, and A/B test results. Let's look at a few example questions:


In [1]:
import json

# Load the test file
with open('data/test.json', 'r') as f:
    test_data = json.load(f)

# Display a few example questions
print("📝 EXAMPLE QUESTIONS FROM THE EXAM\n")
print("="*70)

for section in test_data['sections'][:2]:  # Show first 2 sections
    print(f"\n🔹 Section {section['section']}: {section['title']}")
    print(f"   Dataset: {section['dataset']}")
    print()
    
    # Show first question from each section
    question = section['questions'][0]
    print(f"   Question {question['question_number']} ({question['points']} points):")
    print(f"   {question['question']}")
    print()

print("="*70)


📝 EXAMPLE QUESTIONS FROM THE EXAM


🔹 Section A: E-COMMERCE ANALYSIS
   Dataset: ecommerce_sales.csv

   Question 1 (10 points):
   What is the total revenue generated from the "Electronics" category in Q4 2024? Show your calculation.


🔹 Section B: CUSTOMER SEGMENTATION
   Dataset: customer_data.csv

   Question 5 (10 points):
   How many customers fall into each age group? Young (18-30), Middle (31-50), Senior (51+). Provide exact counts for each segment.



### The LLM's Tools: What It Needs to Access

Here's the critical insight: **the LLM cannot grade these questions from memory alone**. It needs access to:

1. **The Datasets** (`data/datasets/`) - Three CSV files containing the actual data
2. **Grading Rubric** (`data/class_resources/grading_rubric.json`) - Defines how to grade
3. **Ground Truth Answers** (`data/class_resources/ground_truth_answers.json`) - Contains correct answers

Without these tools, the LLM would just be guessing. Let's preview the datasets:


## Prompt Engineering: From Vague to Precise

A "wow-effect" prompt might look like this:

> "Grade this student answer: $6,500"

This is terrible for production. The LLM doesn't know what question, what rubric, what data to check, or how to provide feedback.

A **production-ready prompt** implements these key components:

1. **Clear Role Definition** - WHO the LLM is and WHAT expertise it has
2. **System vs User Messages** - System = standing instructions, User = specific task
3. **Explicit Rules with Chain-of-Thought** - Step-by-step reasoning triggers
4. **Few-Shot Examples** - Show the LLM correct grading examples

We've created all of this in `prompt.py`. Let's examine each component:


In [2]:
# Load and display our prompt structure
from prompt import SYSTEM_PROMPT, USER_PROMPT_TEMPLATE, FEW_SHOT_EXAMPLES

print("SYSTEM PROMPT (Role + Rules)")
print("="*70)
print(SYSTEM_PROMPT)

print("\nUSER PROMPT TEMPLATE (Specific Task)")
print("="*70)
print(USER_PROMPT_TEMPLATE)


SYSTEM PROMPT (Role + Rules)
You are an expert Data Science instructor and grader with 10+ years of experience evaluating student work.

Your role is to grade student exam submissions fairly, accurately, and with detailed feedback. You must:

1. Be objective and consistent in your grading
2. Always verify calculations against the actual datasets
3. Award partial credit for correct methodology even if the final answer is wrong
4. Provide specific, actionable feedback that helps students learn
5. Reference exact data points when explaining errors

CRITICAL RULES:

Rule 1 - ALWAYS Access the Data
You MUST access the actual CSV datasets to verify student calculations. Never guess or estimate. If a student claims "Revenue was $6,500", you must query the dataset to confirm the actual value.

Rule 2 - Follow the Grading Rubric
Access the grading_rubric.json file for each question. It specifies:
- Point allocation (correct answer, showing work, interpretation)
- Partial credit criteria
- Commo

## Structured Output: From Free Text to Validated Objects

After prompt engineering, the next critical step is **forcing structured output**. 

### The Problem with Free Text

A "wow-effect" LLM might return something like:

```
The student got 7 out of 10. Their answer was close but not quite right.
They made some mistakes with the data.
```

This is useless for production because:
- ❌ Can't parse programmatically
- ❌ No point breakdown
- ❌ Missing which errors specifically
- ❌ Can't aggregate across questions
- ❌ No type safety or validation

### The Solution: Pydantic Schemas

We use **Pydantic models** to enforce a strict output structure. The LLM must return validated JSON that matches our schema.


In [3]:
# Let's look at our Pydantic schema for grading results
from schemas import GradingResult

# Display the schema
print("📋 GRADING RESULT SCHEMA")
print("="*70)
print("\nRequired fields:")
for field_name, field in GradingResult.model_fields.items():
    required = "✓" if field.is_required() else "○"
    print(f"  {required} {field_name}: {field.annotation}")
    if field.description:
        print(f"      → {field.description}")
print("\n" + "="*70)


📋 GRADING RESULT SCHEMA

Required fields:
  ✓ question_number: <class 'int'>
      → Question number (1-10)
  ✓ points_earned: <class 'float'>
      → Points earned out of 10
  ○ points_possible: <class 'int'>
      → Maximum points for this question
  ✓ is_correct: <class 'bool'>
      → Whether the answer is fully correct
  ✓ student_answer: <class 'str'>
      → The student's submitted answer
  ✓ correct_answer: <class 'str'>
      → The correct answer
  ✓ points_breakdown: <class 'dict'>
      → Points breakdown: correct_answer, showing_work, interpretation
  ○ error_type: typing.Optional[typing.Literal['wrong_calculation', 'wrong_methodology', 'missing_data', 'incomplete_work', 'no_error']]
      → Type of error if incorrect
  ○ specific_errors: typing.List[str]
      → List of specific errors found (e.g., 'Missed orders ORD020, ORD021')
  ○ what_was_correct: typing.List[str]
      → What the student did correctly (for partial credit)
  ✓ feedback: <class 'str'>
      → Detailed f

### Key Features of Our Schema

**1. Type Safety**
```python
points_earned: float = Field(..., ge=0, le=10)  # Must be 0-10
question_number: int = Field(..., ge=1, le=10)  # Must be 1-10
```

**2. Validation Rules**
```python
@validator('points_earned')
def validate_points(cls, v, values):
    if v > values.get('points_possible', 10):
        raise ValueError('Points earned cannot exceed points possible')
```

**3. Structured Breakdown**
- `points_breakdown`: Dict with correct_answer, showing_work, interpretation
- `specific_errors`: List of exact mistakes
- `data_references`: Which data was checked

**4. Computed Properties**
```python
def get_percentage(self) -> float:
    return (self.points_earned / self.points_possible) * 100
```

Let's create an example:


In [4]:
# Create a valid grading result
result = GradingResult(
    question_number=1,
    points_earned=3.0,
    points_possible=10,
    is_correct=False,
    student_answer="$6,500",
    correct_answer="$7,398.53",
    points_breakdown={
        "correct_answer": 0,
        "showing_work": 1,
        "interpretation": 2
    },
    error_type="wrong_calculation",
    specific_errors=[
        "Missed December orders: ORD020, ORD021, ORD023, ORD025, ORD027, ORD028, ORD030",
        "Incorrect total - should be $7,398.53, not $6,500"
    ],
    what_was_correct=[
        "Correctly identified Q4 as Oct-Dec 2024",
        "Attempted to filter by Electronics category"
    ],
    feedback="""Your answer of $6,500 is incorrect. The correct answer is $7,398.53.

You were on the right track identifying Q4 2024, but you missed 7 orders in December. 
Looking at ecommerce_sales.csv, the December Electronics orders (ORD020, ORD021, ORD023, 
ORD025, ORD027, ORD028, ORD030) total $2,449.90, which you didn't include.

Make sure to check ALL three months when filtering for Q4.""",
    data_references=["ecommerce_sales.csv rows 20-30"]
)

print("✅ Structured Grading Result:")
print(result.to_display_format())


✅ Structured Grading Result:

Question 1: 3.0/10 points (30.0%)

Student Answer: $6,500
Correct Answer: $7,398.53

Points Breakdown:
  - Correct Answer: 0/6
  - Showing Work: 1/2
  - Interpretation: 2/2

Your answer of $6,500 is incorrect. The correct answer is $7,398.53.

You were on the right track identifying Q4 2024, but you missed 7 orders in December. 
Looking at ecommerce_sales.csv, the December Electronics orders (ORD020, ORD021, ORD023, 
ORD025, ORD027, ORD028, ORD030) total $2,449.90, which you didn't include.

Make sure to check ALL three months when filtering for Q4.



### Why This Matters

**Wow-effect approach:**
```python
response = llm.generate("Grade this answer: $6,500")
# Returns: "The student got 3/10 because they missed some data..."
# Now what? Parse text? Hope format is consistent? Good luck!
```

**Production approach:**
```python
result = GradingResult(...)  # Pydantic validates everything
print(result.points_earned)  # 3.0 (guaranteed float)
print(result.get_percentage())  # 30.0% (computed)
print(result.specific_errors)  # List[str] (guaranteed list)
```

Benefits:
✅ **Type-safe** - points_earned is always a float  
✅ **Validated** - Can't have 11/10 points (validator prevents it)  
✅ **Parseable** - JSON in, Python object out  
✅ **Aggregatable** - Easy to sum across questions  
✅ **Database-ready** - Can save directly to DB  
✅ **API-ready** - Can return as JSON response

Now let's integrate this with CrewAI to actually generate these structured outputs from an LLM.


### Aligning Prompt with Schema

This is critical: **your prompt must tell the LLM to output in the exact format your schema expects**.

We've updated `prompt.py` to include:

1. **OUTPUT FORMAT specification in the system prompt** - Shows the exact JSON structure
2. **Few-shot examples with JSON outputs** - Demonstrates the complete structure
3. **User prompt reminder** - "Return your grading result as a valid JSON object"

Let's see the updated prompt with JSON format specification:


In [5]:
# Show the updated prompt with JSON format
from prompt import SYSTEM_PROMPT, FEW_SHOT_EXAMPLES
import json

# Show just the OUTPUT FORMAT section
output_format_section = SYSTEM_PROMPT.split("OUTPUT FORMAT:")[1] if "OUTPUT FORMAT:" in SYSTEM_PROMPT else "Not found"

print("🎯 OUTPUT FORMAT SPECIFICATION IN PROMPT")
print("="*70)
print("OUTPUT FORMAT:" + output_format_section)
print("\n" + "="*70)

# Show one example with JSON output
print("\n📝 FEW-SHOT EXAMPLE WITH JSON OUTPUT")
print("="*70)
example = FEW_SHOT_EXAMPLES[0]
print(f"Question: {example['question']}")
print(f"Student Answer: {example['student_answer']}\n")
print("Expected JSON Output:")
print(json.dumps(example['correct_grading']['json_output'], indent=2))


🎯 OUTPUT FORMAT SPECIFICATION IN PROMPT
OUTPUT FORMAT:
You MUST return your grading result as a JSON object with the following structure:
{
    "question_number": int,
    "points_earned": float,
    "points_possible": int,
    "is_correct": bool,
    "student_answer": str,
    "correct_answer": str,
    "points_breakdown": {
        "correct_answer": float,  // 0-6 points
        "showing_work": float,     // 0-2 points
        "interpretation": float    // 0-2 points
    },
    "error_type": str | null,  // One of: "wrong_calculation", "wrong_methodology", "missing_data", "incomplete_work", "no_error"
    "specific_errors": [str],  // List of specific mistakes found
    "what_was_correct": [str], // List of things student did correctly
    "feedback": str,           // Detailed feedback (minimum 50 characters)
    "data_references": [str]   // Specific data points checked (e.g., "ecommerce_sales.csv rows 20-30")
}


📝 FEW-SHOT EXAMPLE WITH JSON OUTPUT
Question: What is the total reve

### Why Prompt-Schema Alignment is Critical

**Before (misaligned):**
```
Prompt: "Grade this and provide feedback"
LLM: "The student got 7/10 because..."
You: "Great, now how do I parse this text?"
```

**After (aligned):**
```
Prompt: "Return JSON with these exact fields: question_number, points_earned, points_breakdown..."
LLM: Returns valid JSON matching GradingResult schema
You: result = GradingResult(**json_response)  # ✅ Works perfectly
```

The LLM now knows to output:
✅ All required fields (question_number, points_earned, etc.)  
✅ Correct data types (float for points, bool for is_correct)  
✅ Proper structure (points_breakdown as dict, specific_errors as list)  
✅ Valid JSON that Pydantic can parse and validate

This alignment is **essential** for production systems. Without it, you're parsing free text and hoping for consistency.


## Tools: Giving the LLM Access to Data

Now comes the critical part: **tool integration**. Without tools, the LLM can't verify student answers against actual data.

### Why Tools Matter

Remember our prompt tells the LLM:
- "Access the grading rubric"
- "Access ground truth answers"  
- "Access the dataset to verify calculations"

But HOW does the LLM do this? Through **tools** (also called functions or function calling).

### Our Tool Arsenal

We've created 6 tools in `tools.py`:

1. **`get_grading_rubric(question_number)`** - Retrieves grading criteria
2. **`get_ground_truth_answer(question_number)`** - Gets correct answer & methodology
3. **`read_dataset(filename, num_rows)`** - Reads CSV files
4. **`query_dataset(filename, filters, columns, calculate)`** - Filters and aggregates data
5. **`calculate_revenue(filename, filters)`** - Calculates sales revenue
6. **`get_dataset_info(filename)`** - Shows dataset metadata

Let's see them in action:


In [1]:
from tools import GradingTools

# Initialize tools
tools = GradingTools()

# Tool 1: Get grading rubric
print("Tool 1: get_grading_rubric(1)")
print("="*70)
rubric = tools.get_grading_rubric(1)
print(f"Question: {rubric['title']}")
print(f"Points: {rubric['points']}")
print(f"Correct Answer: {rubric['correct_answer']}")
print(f"\nFull Credit Criteria:")
for criterion in rubric['full_credit_criteria']:
    print(f"  - {criterion}")

print("\n\nTool 2: get_ground_truth_answer(1)")
print("="*70)
truth = tools.get_ground_truth_answer(1)
print(f"Correct Answer: {truth['correct_answer']}")
print(f"Methodology: {truth['methodology']}")
print(f"Dataset Used: {truth['dataset_used']}")


Tool 1: get_grading_rubric(1)
Question: Total Revenue from Electronics in Q4 2024
Points: 10
Correct Answer: $7,398.53

Full Credit Criteria:
  - Correct total: $7,398.53
  - Shows filtering for Electronics category AND Q4 dates (Oct-Dec 2024)
  - Shows summation method (even if using tool/code)


Tool 2: get_ground_truth_answer(1)
Correct Answer: $7,398.53
Methodology: Filter for category='Electronics' AND date between 2024-10-01 and 2024-12-31, then sum (quantity × unit_price)
Dataset Used: ecommerce_sales.csv


In [10]:
# Tool 3 & 4: Query dataset to verify student work
print("Tool 3: query_dataset() - Filter Electronics orders")
print("="*70)
result = tools.query_dataset(
    'ecommerce_sales.csv',
    filters={'category': 'Electronics'},
    calculate='count'
)
print(result)

print("\n\nTool 4: calculate_revenue() - Calculate Q4 Electronics revenue")
print("="*70)
# For Q4, we'd need date filtering - let's just show Electronics
revenue_result = tools.calculate_revenue(
    filters={'category': 'Electronics'}
)
# Show just the first few lines
print('\n'.join(revenue_result.split('\n')[:6]))


Tool 3: query_dataset() - Filter Electronics orders
Count: 17 records


Tool 4: calculate_revenue() - Calculate Q4 Electronics revenue
Revenue Calculation:
Number of orders: 17
Total revenue: $9,149.76

Breakdown by order:
   order_id  quantity  unit_price  revenue


### How the LLM Uses Tools

Here's the flow when grading Question 1:

1. **LLM reads the prompt**: "Grade this student answer: $6,500"
2. **LLM thinks**: "I need to verify this. Let me use tools..."
3. **LLM calls `get_ground_truth_answer(1)`**: Gets correct answer = $7,398.53
4. **LLM calls `query_dataset('ecommerce_sales.csv', filters={'category': 'Electronics'})`**: Sees there are 17 Electronics orders
5. **LLM calls `calculate_revenue(filters={'category': 'Electronics'})`**: Calculates actual revenue = $7,398.53
6. **LLM compares**: Student said $6,500, actual is $7,398.53 → WRONG
7. **LLM returns structured JSON**: Points earned, specific errors, feedback

### Wow-Effect vs Production

**Wow-effect approach:**
```python
# LLM has no tools, just guesses
llm.generate("Is $6,500 correct for Q4 Electronics revenue?")
# Returns: "That seems reasonable for a quarter's revenue"
# 😱 WRONG! No way to verify!
```

**Production approach:**
```python
# LLM uses tools to verify against actual data
llm.generate(prompt, tools=[
    get_grading_rubric,
    get_ground_truth_answer,
    query_dataset,
    calculate_revenue
])
# LLM calls calculate_revenue() → Gets $7,398.53
# Returns: "Incorrect. Student said $6,500, actual is $7,398.53"
# ✅ VERIFIED against real data!
```

This is the power of tool integration: **the LLM can fact-check itself**.
